In [1]:
#%pip install pandas
# %pip uninstall TA-Lib
# %pip uninstall numpy


# filepath: c:\Users\mahme\trading_bot\notebooks\train_lstm.ipynb
import os
import sys
sys.path.append(os.path.abspath(".."))

In [2]:
os.chdir("..")


In [3]:
import pandas as pd

from src.data_fetch import connect_mt5, fetch_symbol_data
from src.features import (add_RSI_EMA, add_ATR, add_candlestick_patterns,
                          add_VSA_signals_refined, detect_order_blocks,create_sequences)


# 1. Fetch or load CSV
df = pd.read_csv("data/XAUUSD_M5.csv", parse_dates=["time"])


In [4]:

# ----- Cell: Compute Indicators -----
import talib

df = pd.read_csv("data/XAUUSD_M5.csv", parse_dates=["time"])
df = add_RSI_EMA(df, rsi_period=14, ema_periods=[20, 50])
df = add_ATR(df, atr_period=14)

# 1. Compute per-bar 14-period ATR (if not already present)
df["ATR_14"] = talib.ATR(df["High"], df["Low"], df["Close"], timeperiod=14)

# 2. Compute next bar’s percentage change
df["next_pct_change"] = (df["Close"].shift(-1) - df["Close"]) / df["Close"]

# 3. Define threshold = 0.5 × (ATR / Close)
df["threshold"] = 0.5 * (df["ATR_14"] / df["Close"])

# 4. Define a function to label Up/Down/Hold
def label_with_hold(row):
    change = row["next_pct_change"]
    thr    = row["threshold"]
    if abs(change) <= thr:
        return 2   # Hold (no significant move)
    elif change > thr:
        return 1   # Up
    else:
        return 0   # Down

# 5. Apply the function to create a 3-class label
df["Direction3"] = df.apply(label_with_hold, axis=1)
# -----------------------------------------

df = add_candlestick_patterns(df)
df = add_VSA_signals_refined(df)
df = detect_order_blocks(df)
# -----------------------------------------


In [5]:
# -----------------------------------------
# 2.3: Load H1 and compute its RSI, EMA, ATR
df_h1 = pd.read_csv("data/XAUUSD_H1.csv", parse_dates=["time"])

# 2.3.1 Compute H1 indicators exactly as for M5
df_h1 = add_RSI_EMA(df_h1,  rsi_period=14, ema_periods=[50, 100])
df_h1 = add_ATR(df_h1,     atr_period=14)

# 2.3.2 Keep only the columns we need from H1
df_h1 = df_h1[["time", "RSI", "EMA_50", "ATR"]].rename(columns={
    "RSI": "H1_RSI",
    "EMA_50": "H1_EMA_50",
    "ATR": "H1_ATR"
})

# 2.3.3 For each M5 bar, find the most recent H1 bar (floor to the hour)
# Create a column that floors M5 time to the preceding hour
df["H1_timefloor"] = df["time"].dt.floor("H")

# 2.3.4 Merge H1 onto M5 by matching the floored time
df = df.merge(
    df_h1,
    left_on="H1_timefloor",
    right_on="time",
    how="left",
    suffixes=("", "_h1")
)

# 2.3.5 Clean up: drop the extra columns
df.drop(columns=["time_h1", "H1_timefloor"], inplace=True)
# -----------------------------------------


C:\Users\mahme\AppData\Local\Temp\ipykernel_47896\2090988042.py:18: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["H1_timefloor"] = df["time"].dt.floor("H")


In [6]:
# -----------------------------------------
# 2.4: Quick sanity check
print("Columns after H1 merge:", df.columns.tolist())
print("Sample rows with H1 features:")
display(df[["time", "Close", "H1_RSI", "H1_EMA_50", "H1_ATR"]].head(8))
print("NaN counts (should be minimal after dropping):")
print(df[["H1_RSI", "H1_EMA_50", "H1_ATR"]].isna().sum())
# -----------------------------------------


Columns after H1 merge: ['time', 'Open', 'High', 'Low', 'Close', 'Volume', 'RSI', 'EMA_20', 'EMA_50', 'ATR', 'ATR_14', 'next_pct_change', 'threshold', 'Direction3', 'HAMMER', 'ENGULFING', 'DOJI', 'vsa_signal', 'VSA_No_Demand', 'VSA_No_Supply', 'VSA_Buying_Climax', 'VSA_Selling_Climax', 'VSA_Stopping_Volume', 'OB_type', 'OB_price', 'H1_RSI', 'H1_EMA_50', 'H1_ATR']
Sample rows with H1 features:


,time,Close,H1_RSI,H1_EMA_50,H1_ATR
0,2025-02-21 08:10:00,2928.79,42.792434,2934.764935,8.449047
1,2025-02-21 08:15:00,2927.93,42.792434,2934.764935,8.449047
2,2025-02-21 08:20:00,2927.89,42.792434,2934.764935,8.449047
3,2025-02-21 08:25:00,2928.77,42.792434,2934.764935,8.449047
4,2025-02-21 08:30:00,2930.80,42.792434,2934.764935,8.449047
5,2025-02-21 08:35:00,2930.33,42.792434,2934.764935,8.449047
6,2025-02-21 08:40:00,2929.42,42.792434,2934.764935,8.449047
7,2025-02-21 08:45:00,2929.48,42.792434,2934.764935,8.449047


NaN counts (should be minimal after dropping):
H1_RSI       0
H1_EMA_50    0
H1_ATR       0
dtype: int64


In [7]:
# -----------------------------------------
# 2.5: Drop any NaNs (including H1 columns) and reset index
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

# 2.5.1: Inspect new 3-class distribution (optional; may be unchanged)
dist3     = df["Direction3"].value_counts(normalize=True) * 100
print("Post-H1-merge 3-class distribution (%):")
print(dist3)

# 2.5.2: Split into train/val chronologically (80/20)
train_size = int(len(df) * 0.8)
df_train   = df.iloc[:train_size].copy()
df_val     = df.iloc[train_size:].copy()

Post-H1-merge 3-class distribution (%):
Series([], Name: proportion, dtype: float64)


In [8]:
# 2.5.3: Re-define feature_cols (add H1 features)
feature_cols = [
    "Close", "RSI",       "EMA_20",   "EMA_50",  "ATR",
    "H1_RSI", "H1_EMA_50","H1_ATR",
    # ... plus whatever VSA/OB one-hot columns you have, e.g.:
    "VSA_No_Demand", "VSA_No_Supply", "VSA_Buying_Climax",
    "VSA_Selling_Climax", "VSA_Stopping_Volume",
    "OB_bullish", "OB_bearish", "Distance_to_OB"
]

label_col = "Direction3"
lookback  = 60

# 2.5.4: Re-create sequences with new features
X_train, y_train, scaler = create_sequences(df_train, feature_cols, label_col, lookback)
X_val,   y_val,   _      = create_sequences(df_val,   feature_cols, label_col, lookback)

print("New shapes including H1 features:")
print("X_train:", X_train.shape, "y_train:", y_train.shape)
print("X_val  :", X_val.shape,   "y_val  :", y_val.shape)

KeyError: "['OB_bullish', 'OB_bearish', 'Distance_to_OB'] not in index"

In [9]:

lookback = 60
X_train, y_train, scaler = create_sequences(df_train, feature_cols, label_col, lookback)
X_val,   y_val,   _      = create_sequences(df_val,   feature_cols, label_col, lookback)

print("X_train shape:", X_train.shape, "y_train shape:", y_train.shape)
print("X_val   shape:", X_val.shape,   "y_val   shape:", y_val.shape)


X_train shape: (15927, 60, 5) y_train shape: (15927,)
X_val   shape: (3936, 60, 5) y_val   shape: (3936,)


In [10]:
# ----- New Model Cell -----
# 1) Import our build function
from src.model_lstm import build_3class_lstm_model

# 2) Define input_shape based on X_train
#    X_train.shape == (num_samples, lookback, num_features)
lookback     = X_train.shape[1]
num_features = X_train.shape[2]
input_shape  = (lookback, num_features)

# 3) Build and compile
model = build_3class_lstm_model(input_shape)
model.summary()

# 4) (Optional) If you want to save a reference to the best model:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
callbacks = [
    EarlyStopping(monitor="val_accuracy", patience=5, restore_best_weights=True),
    ModelCheckpoint("models/lstm_best.h5", monitor="val_accuracy", save_best_only=True)
]


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 60, 128)        │        68,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 60, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 60, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 121,091 (473.01 KB)

 Trainable params: 120,643 (471.26 KB)

 Non-trainable params: 448 (1.75 KB)

In [11]:
# ----- New Training Cell -----
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,            # or more, with EarlyStopping
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)


Epoch 1/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.5822 - loss: 1.1734

249/249 ━━━━━━━━━━━━━━━━━━━━ 18s 62ms/step - accuracy: 0.5823 - loss: 1.1731 - val_accuracy: 0.6087 - val_loss: 1.0772
Epoch 2/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 13s 53ms/step - accuracy: 0.6046 - loss: 1.0664 - val_accuracy: 0.6087 - val_loss: 1.0486
Epoch 3/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 14s 55ms/step - accuracy: 0.6041 - loss: 1.0427 - val_accuracy: 0.6087 - val_loss: 1.0244
Epoch 4/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 14s 56ms/step - accuracy: 0.6063 - loss: 1.0195 - val_accuracy: 0.6087 - val_loss: 1.0025
Epoch 5/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 14s 57ms/step - accuracy: 0.6005 - loss: 1.0019 - val_accuracy: 0.6087 - val_loss: 0.9834
Epoch 6/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 15s 59ms/step - accuracy: 0.6041 - loss: 0.9826 - val_accuracy: 0.6087 - val_loss: 0.9678
